# Preparar Dataset

Este notebook se encarga de extrar la información almacenada en los Excel de la carpeta `data/GAM/Result ResArtEmotion`. El resultado será un fichero json con los datos personales de los usuarios, y otro fichero json con las respuestas que ha hecho cada usuario a las obras que se muestran en la encuesta.

In [13]:
import pandas as pd

In [14]:
# Diccionarios
relationship_with_art = {
    # EN
    'My job is related to the art world': 3,
    'I am passionate about the art': 2,
    'I am a little interested in art': 1,
    'I am not interested in art': 0,
    # ES
    'Mi trabajo está relacionado con el mundo del arte': 3,
    'Me apasiona el arte': 2,
    'Me interesa un poco el arte': 1,
    'No me interesa el arte': 0,
    #FI
    'Työskentelen taiteen ja kulttuurin parissa.': 3,
    'Rakastan taidetta.': 2,
    'Olen jonkin verran kiinnostunut taiteesta.': 1,
    'En ole kiinnostunut taiteesta': 0,
    # HE
    'התפקיד שלי קשור לעולם האמנות': 3,
    'אני נלהב מהאמנות': 2,
    'אני קצת מתעניין באמנות': 1,
    'אני לא מתעניין באמנות': 0,
    # IT
    'Il mio lavoro è legato al mondo dell\'arte': 3,
    'Sono appassionato/a di arte': 2,
    'Sono un po\' interessato/a all\'arte': 1,
    'Non sono interessato all\'arte': 0
}

visit_museums = {
    # EN
    'I like to visit museums frequently': 2,
    'I go occasionally to museums or art exhibitions': 1,
    'I rarely visit museums or art exhibitions': 0,
    # ES
    'Me gusta visitar museos con frecuencia': 2,
    'Voy de vez en cuando a museos o exposiciones de arte': 1,
    'Pocas veces voy a museos o exposiciones de arte': 0,
    # FI
    'Käyn usein museoissa.': 2,
    'Käyn silloin tällöin museoissa ja taidenäyttelyissä.': 1,
    'Käyn vain harvoin museoissa tai taidenäyttelyissä.': 0,
    # HE
    'אני אוהב לבקר במוזיאונים לעיתים קרובות': 2,
    'אני הולך מדי פעם למוזיאונים או לתערוכות אמנות': 1,
    'לעתים נדירות אני מבקר במוזיאונים או בתערוכות אמנות': 0,
    # IT
    'Mi piace visitare spesso i musei': 2,
    'Vado occasionalmente a musei o mostre d\'arte': 1,
    'Visito raramente musei o mostre d\'arte': 0,
}

languages = ['EN', 'ES', 'FI', 'HE', 'IT']

In [15]:
def change_art_relationship(row):
    if isinstance(row, str):
        options = row.split(', ')
        return [relationship_with_art[o] for o in options]
    else:
        return []

In [22]:
# Cargamos los datos de los cuadros
artworks_df = pd.read_json('../../data/GAM/artworks.json')

# Creamos lista de usuarios y de interacciones vacio
users = []

In [23]:
id_user = 1
interactions = list()

for lang in range(len(languages)):

    path = '../../data/GAM/answers/{}.xlsx'.format(languages[lang])
    personal_data_df = pd.read_excel(path, sheet_name="Risposte del modulo 1")

    for row in range(len(personal_data_df)):
        r = personal_data_df.iloc[row,:]

        if isinstance(r['How would you define your relationship with Art?'], str) and isinstance(r['Do you like going to museums or art exhibitions?'], str):
            user = {
                'id': id_user,
                'gender': r['Gender'],
                'age': r['Age'],
                'relationship_art': [relationship_with_art[o] for o in r['How would you define your relationship with Art?'].split(', ')],
                'like_museums': visit_museums[r['Do you like going to museums or art exhibitions?']],
                'languages': languages[lang]
            }
            users.append(user)

            id_user += 1

            first_interaction = 4
            for art in range(1,13):
                start_index = ((art - 1) * 5) + 4
                end_index = start_index + 5

                interaction = {
                    'user': id_user,
                    'artwork': art
                }

                for answer in range(start_index, end_index):
                    q = 'q{}'.format((answer % 5) + 1)
                    interaction[q] = r.iloc[answer]

                interactions.append(interaction)

In [26]:
pd.DataFrame(users).to_csv('../../data/GAM/clean_data/users.csv', index=False)

In [31]:
pd.DataFrame(interactions).to_csv('../../data/GAM/clean_data/interactions.csv', index=False)

In [33]:
artworks_df.to_csv('../../data/GAM/clean_data/artwors.csv', index=False)